# I See Dead People: An Analysis

Bryan Bumgardner, Data Scientist  
February - March 2016

<img src="images/michaelguns.gif">

### We love watching violence.   
That being said, it's worth studying how it fits in our popular culture. During the Metis Data Science bootcamp (shameless plug), while working on another project I discovered bodycounters.com[http://www.bodycounters.com/]. The dedicated volunteers at this site count and categorize the number of deaths in movies, and to date, have counted for over 2,000 movies. Go grab some beers, count the dead people in your favorite movie, and contribute to the site. 

The data are sitting on their site for anyone to see, and it gave me an idea. I reached out to the site and had some correspondence with Dana, who was gracious enough to share a CSV of all the data. I then cross referenced this data with information from another site, thenumbers.com, which shares basic budget and financial information about thousands of movies. I took this data, mashed it together, and asked some questions. 

Thanks again to the volunteers at bodycounters. Y'all are the greatest. Below is a quick data science passion project to analyze and find patterns in this great data you've collected. 

### Major questions:
1. Is the amount of on-screen deaths going up in modern movies?

2. Does a movie's death count have an effect on that movie's domestic gross?
3. How does budget factor into a movie's death count?
4. Is there a relationship between the movie's MPAA rating and body count?
5. What about death count and genre?


### Table of Contents:
1. Reading and combining the data
2. Outputting various data frames
3. Visualizing
4. Conclusions
5. Documentation of the data

In [1]:
import pandas as pd 
import numpy as np
import csv
import os
from datetime import datetime
from future.builtins import next
import re
import logging
import optparse

from difflib import SequenceMatcher

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression 
from patsy import dmatrices

import matplotlib.pyplot as plt, mpld3
import seaborn as sns

%matplotlib inline
sns.set(color_codes=True)

### Step 1: read in and clean the data

Deaths data is a csv, provided by the wonderful folks at BodyCounters. 

In [2]:
deaths = pd.read_csv("data/moviesmovies.csv", encoding="latin-1", index_col="movieskey") #might as well make use of their key

deaths["moviename"] = deaths["moviename"].str.rstrip("\t").str.lower() #cleaning some trailing spaces and standardizing for the matching
deaths["deathcomment"] = deaths["deathcomment"].str.rstrip("\t")
deaths["moviename"] = deaths["moviename"].str.replace(".", '')
deaths["moviename"] = deaths["moviename"].str.replace(",", '')
deaths["moviename"] = deaths["moviename"].str.replace("/", '')
deaths["moviename"] = deaths["moviename"].str.replace("'", '')
deaths = deaths.sort(columns="moviename") #it will match faster if we sort these in alphabetical order

deaths.to_csv("deathscleaned.csv", encoding='utf-8') #sending it back out for matching later in OpenRefine

/Users/bryanbumgardner/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Let's scrape all the movie data - EVER - because we are crazy. And also because I'm not interested in paying for access to their database. 

So what you see below is me scraping all the budgets and domestic box offices for every movie in the alphabetized database. Every letter has a chart with all the movies starting with that letter. I downloaded each webpage from the data source as an HTML webpage then pulled the charts from that, which is less likely to get you banned than hitting up their site with Selenium.

I then mashed the charts together into one GIANT dataframe which will be helpful for record linkage. This needs properly cleaned as well.

In [3]:
path = 'movie_pages/' #where the HTML files are
listing = os.listdir(path) #read the pages from the directory
all_movie_data = pd.DataFrame(columns = ['Release Date', 'Movie', 'Genre', 'ProductionBudget','DomesticBox Officeto Date', 'Trailer'])
# empty dataframe waiting for all the goodies

for i, infile in enumerate(listing): #iterate through the files in the directory 'path'
    temp_list = pd.read_html("movie_pages/"+infile, header=0) #THANK YOU BASED PANDAS. Cleans HTML away and pulls out just the tables.
    temp_frame = pd.DataFrame(temp_list[0]) #temp_list is a list of dataframes. Take the first (and only) one
    all_movie_data = all_movie_data.append(temp_frame) #append this to the master dataframe 
    
del all_movie_data['Trailer'] #not necessary and screwed up the dropna

all_movie_data.rename(columns = {'DomesticBox Officeto Date': 'BoxOffice'}, inplace = True)
all_movie_data = all_movie_data.dropna() #drop unneccessary lines with incomplete data

In [4]:
label_movie_data = all_movie_data #Save this just in case. Might try and do something with the labels, and this has the un-cleaned labels. 

In [5]:
all_movie_data = all_movie_data[all_movie_data.BoxOffice != "$0"]
all_movie_data = all_movie_data[all_movie_data.ProductionBudget != "$0"]
# remove all with $0 (I don't know those values got recorded? Negligence on the site, I think.)

#Formatting all the data so it's actually useful and easier to sum, average, and categorize, along with effective timestamps. 

all_movie_data["Movie"] = all_movie_data["Movie"].str.lower()
all_movie_data["Movie"] = all_movie_data["Movie"].str.replace(".", '')
all_movie_data["Movie"] = all_movie_data["Movie"].str.replace(",", '')
all_movie_data["Movie"] = all_movie_data["Movie"].str.replace(")", '')
all_movie_data["Movie"] = all_movie_data["Movie"].str.replace("'", '') # fuck it we'll do it live. Can you clean this up?
all_movie_data["Movie"] = all_movie_data["Movie"].str.replace("(", '')
all_movie_data["Genre"] = all_movie_data["Genre"].str.lower()
all_movie_data["ProductionBudget"] = all_movie_data["ProductionBudget"].str.strip('$')
all_movie_data["ProductionBudget"] = all_movie_data["ProductionBudget"].str.replace(",", '')
all_movie_data["BoxOffice"] = all_movie_data["BoxOffice"].str.strip('$')
all_movie_data["BoxOffice"] = all_movie_data["BoxOffice"].str.replace(",", '')
all_movie_data = all_movie_data.sort(columns="Movie")


/Users/bryanbumgardner/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [6]:
all_movie_data["Release Date"] = pd.to_datetime(all_movie_data["Release Date"], format='%b %d, %Y')
# pulled this out to try and identify where my data was getting corrupted
all_movie_data.to_csv("all_movie_data.csv", encoding='utf-8')
# exporting the clean CSV for record linkage

Ok so I'm really lazy and looked into how well these would match with Excel. Turns out the data is filthy and won't match well at all. There are extreme inconsistencies in movie names from both datasets. So I'll have to do fuzzy matching.

### Step 2. Combine the data using record linkage

So the Pandas dataframe-compatible record linkage package, recordlinkage, is turned off right now. I'm still trying to learn how to do this with something called FuzzyWuzzy, which is what you see below.

However, I used OpenRefine to connect the datasets outside of Python, then I'm reading it back into here as a CSV. This was much faster but doesn't scale well, so a true Python script is needed long term.

I did the quick cleaning by using OpenRefine's built-in matching powers mixed with this great fuzzy string reading service that emphasizes reconciliation:
http://okfnlabs.org/reconcile-csv/

In [ ]:
"""#preparing the target dataframe
merged_movie_data = pd.DataFrame(columns = ['Movie', 'ReleaseDate', 'Genre', 'ProductionBudget', 'BoxOffice', 'DeathCount'])
test_dataframe = pd.DataFrame(columns = ['name_ratio', 'name_token_sort_ratio', 'name_partial_ratio', 'match'])
test_death_count = deaths[:100]
test_movie_data = all_movie_data[:100] #gonna see how this works
"""

#This is all under construction!!

In [ ]:
"""for row in test_movie_data.iterrows():
    current_movie_name = test_movie_data["Movie"]
    
    
        for row2 in test_death_count.iterrows():
            second_movie_name = test_death_count["moviename"]
            second_movie_death_count = test_death_count["deathcount"]
            
            
    m = SequenceMatcher(None, current_movie_name, second_movie_name)
    if m.ratio() > 96:
        test_movie_data["DeathCount"] = test_movie_data["DeathCount"].append(second_movie_death_count)
        
        test_death_count.iterrows()
row = next(test_death_count.iterrows())

def send_back_first_title(test_movie_data)
    for row in test_movie_data.iterrows():
        current_movie_name = test_movie_data["Movie"]"""

# Skip past this! 

# XTREME VIOLENCE ALERT

<img src="images/bloodies.gif">

I discovered the dataset contained LOTS of speculation, guesstimations of body counts on exploded planets, in destroyed spaceships, etc. Many movies had upwards of 1 billion deaths. Knowing this data isn't representative of personal, on-screen images of death, I removed all movies with more than 100,000 recorded deaths. Just in case, I kept the outliers in a second frame, as I noticed many of them were high-budget blockbusters - that could be relevant.

In [7]:
final_data_frame_minus_outliers = pd.read_csv("finalcleaneddata.csv", encoding="latin-1") #no outliers
final_data_frame_complete = pd.read_csv("untouchedfinaldata.csv", encoding="latin-1") #with outliers

In [8]:
final_data_frame_minus_outliers = final_data_frame_minus_outliers.dropna() # In the pairing process, we had some movies with missing values
final_data_frame_complete = final_data_frame_complete.dropna()

final_data_frame_minus_outliers = final_data_frame_minus_outliers.drop("index", 1)
final_data_frame_complete = final_data_frame_complete.drop("index", 1)
#drop some extra stuff that was only useful during linkage. 

final_data_frame_minus_outliers["ReleaseDate"] = pd.to_datetime(final_data_frame_minus_outliers["ReleaseDate"], format="%m/%d/%Y")
final_data_frame_complete["ReleaseDate"] = pd.to_datetime(final_data_frame_complete["ReleaseDate"], format="%m/%d/%Y")
#fitting to my required datetime needs

In [9]:
pd.options.display.max_rows = 1050 #change this if you ever need to explore the data
len(final_data_frame_minus_outliers) #so 1040 movies minus the outliers. 

1040

In [10]:
len(final_data_frame_complete) #count of all movies.

1060

In [11]:
final_data_frame_complete.head(1)

,moviename,BoxOffice,ProductionBudget,Genre,ReleaseDate,deathcount,deathcomment
4,13 going on 30,57139723,30000000,comedy,2004-04-23,0,0


In [13]:
#We have to export this as a CSV for the D3 visualization later on. 
final_data_frame_complete.to_csv("d3/final_data_frame_complete.csv", encoding='utf-8')
final_data_frame_minus_outliers.to_csv("d3/final_data_frame_minus_outliers.csv", encoding='utf-8')

So we have all our stuff and some extra fun comments that will be interesting during visualization. Only twenty movies were removed during the outlier cleaning. 

### Step 3: Statistical Analysis
Now that we have the data, let's revisit the questions we asked and find the best models to answer them.

### Initial Analysis: what factors influence box office revenue?

In [ ]:
Y, x = dmatrices('BoxOffice ~ ProductionBudget + Genre + deathcount + ReleaseDate', data=final_data_frame_complete, return_type='dataframe')
# This sets BoxOffice as our dependent variable. Everything after the ~ is combined into intercepts as one massive variable.
model = sm.OLS(Y, x)
results = model.fit()
results.summary()

So as you can see, the movie's genre and production budget has more of an effect on the box office than the deathcount, at least in this model.  
Including the release dates (which are all basically unique) greatly improves the P values of the other variables.
### Insight into the genres  
Movies with better chances of guaranteed high profit: adventure, drama, horror, thriller/suspense.
Movies with highly variable chances for high profit: black comedy, comedy
Movies with unclear outcomes: Romantic comedy, westerns
Movies that don't do well: Musicals

In [ ]:
# write out in better detail why this is happening. 

### Is the amount of on-screen deaths going up in modern movies?   
Technique to solve this: create an average of deaths per year in movies. 



In [ ]:
average_deaths = final_data_frame_minus_outliers[["ReleaseDate", "deathcount", "Genre"]] #taking Genre for visualization. I have a hunch...

In [ ]:
average_deaths["Year"] = average_deaths["ReleaseDate"].map(lambda x: x.strftime('%Y'))
# strip just the year from the datetime object and put it into it's own column
# This is a deep copy warning, it's not an error. Ignore this for now, it's not causing a problem. 

A quick statistical test to see if year has a big effect on movie count. Averages aren't great, but this is just a top level view. 

In [ ]:
cleaned_average_deaths = average_deaths.groupby(['Year']).mean().reset_index() #group by the year, get the mean of the deathcounts,
# and use reset_index to move the year into it's own column instead of using it as the index. Just personal preference.

x = pd.to_numeric(cleaned_average_deaths["Year"]) #for modeling, these types must be numeric. Originally were floats. 
y = pd.to_numeric(cleaned_average_deaths["deathcount"])

model = sm.OLS(x, y) #Linear Regression with Ordinary Least Squares.
results = model.fit()
results.summary()

In [ ]:
#sns.regplot(x="Year", y="deathcount", data=cleaned_average_deaths)
plt.scatter(x,y)

At first glance, it appears that death counts are going up. This requires a little more analysis - perhaps a histogram or box plot. That can happen later.

### What factors affect a movie's death count, then?

In [ ]:
Y, x = dmatrices('deathcount ~ ProductionBudget + Genre + ReleaseDate + BoxOffice', data=final_data_frame_minus_outliers, return_type='dataframe')
# left out Box Office because that number is determined chronologically. The amount of deaths won't change post-production of a movie.
model = sm.OLS(Y, x)
results = model.fit()
results.summary()
# Production budget, obviously

### What affects a movie's production budget?

In [ ]:
Y, x = dmatrices('ProductionBudget ~ deathcount + Genre + ReleaseDate', data=final_data_frame_minus_outliers, return_type='dataframe')
#  Same here. Left out Box Office because that number is determined chronologically.
model = sm.OLS(Y, x)
results = model.fit()
results.summary()
#LOL Some of these genres don't get shit for budgets. 

### Step 4. Visualizing 

In [ ]:
final_data_frame_complete.head(1)

In [ ]:
sns.regplot(x="BoxOffice", y="ProductionBudget", data=final_data_frame_minus_outliers)

In [ ]:
sns.boxplot(x="ProductionBudget", y="Genre", data=final_data_frame_minus_outliers);

In [ ]:
sns.boxplot(x="BoxOffice", y="Genre", data=final_data_frame_minus_outliers);

In [ ]:
final_data_frame_complete.head(1)

A final D3 visualization:

### 5. Documentation of the data:

Budget and financial data of movies: http://www.the-numbers.com/movie/budgets/all